In [1]:
! codecarbon init


Welcome to CodeCarbon, here is your experiment id:
8c5eece3-1309-4aea-a822-e8f636025071

CodeCarbon automatically added this id to your local config: ./.codecarbon.config



In [1]:
import os
from experiment_setup import setups
import torch
from codecarbon import EmissionsTracker
from datetime import datetime
import json
from IPython.display import display, HTML
from carbontracker.tracker import CarbonTracker
import subprocess



In [2]:
def track_training_C02_emissions_carbontracker(command, trained_model_folder):
    # Inicializamos el tracker
    tracker = CarbonTracker(epochs=1)  # Usamos epochs=1 para rastrear toda la ejecución como una única época

    try:
        # Obtenemos la fecha y hora de inicio
        start_time = datetime.now()

        # Iniciamos el tracker
        tracker.epoch_start()

        # Ejecutamos el comando de entrenamiento
        training_process = subprocess.run(command, shell=True, capture_output=True, text=True)

        # Detenemos el tracker y obtenemos las emisiones finales
        tracker.epoch_end()
        tracker.stop()  # Detenemos el tracker al finalizar

        # Obtenemos la fecha y hora de finalización
        end_time = datetime.now()

        # Imprimimos la salida de la ejecución
        print(f"Salida de STDOUT: {training_process.stdout}")

        # Obtén las emisiones de CO2
        emissions = tracker.final_emissions

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None

    except Exception as e:
        print(f"Unexpected Error: {e}")
        return None

    # Ruta del archivo JSON
    json_file_path = os.path.join("..", "trained_models", trained_model_folder, "trainingData.json")

    # Leer el archivo JSON existente
    existing_data = []
    if os.path.exists(json_file_path):
        try:
            with open(json_file_path, 'r') as f:
                existing_data = json.load(f)
        except json.JSONDecodeError:
            print(f"El archivo {json_file_path} está vacío o contiene datos inválidos, se inicializará como una lista vacía.")
            existing_data = []

    # Preparar la información del entrenamiento
    training_info = {
        "training_iteration": len(existing_data) + 1,  # Número de iteración basado en el tamaño del dataset existente
        "date": start_time.strftime("%Y-%m-%d %H:%M:%S"),
        "execution_time_seconds": (end_time - start_time).total_seconds(),
        "CO2_emissions_kg": emissions
    }

    # Agregar la nueva información del entrenamiento
    existing_data.append(training_info)

    # Escribir los datos actualizados al archivo JSON
    with open(json_file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

    # Liberar memoria
    gc.collect()

    # Finalmente, retornamos las emisiones de CO2
    return emissions

In [3]:
# Función para entrenar un modelo, rastrear las emisiones de CO2 y guardar la información de entrenamiento
def track_training_C02_emissions(command, trained_model_folder, lossFunction, dataset):

    # Inicializamos el tracker
    tracker = EmissionsTracker()
    
    try:
        # Obtenemos la fecha y hora de inicio
        start_time = datetime.now()

        #iniciamos el tracker
        tracker.start()
        
        # Ejecutamos el comando de entrenamiento
        training_process = subprocess.run(command, shell=True, capture_output=True, text=True)

        # Detenemos el tracker y obtenemos las emisiones finales
        emissions = tracker.stop()

        # Obtenemos la fecha y hora de finalización
        end_time = datetime.now()

        # Imprimimos la salida de la ejecución
        print(f"Salida de STDOUT: {training_process.stdout}")

    except FileNotFoundError as e:
        print(f"Error: {e}")

    except Exception as e:
        print(f"Unexpected Error: {e}")
    
    

    # Ruta del archivo JSON
    json_file_path = os.path.join("..", "trained_models", trained_model_folder ,"trainingData.json")
    
    # Leer el archivo JSON existente
    existing_data = []
    if os.path.exists(json_file_path):
        try:
            with open(json_file_path, 'r') as f:
                existing_data = json.load(f)
        except json.JSONDecodeError:
            print(f"El archivo {json_file_path} está vacío o contiene datos inválidos, se inicializará como una lista vacía.")
            existing_data = []

    # Preparar la información del entrenamiento
    training_info = {
        "training_iteration": len(existing_data) + 1,  # Número de iteración basado en el tamaño del dataset existente
        "date": start_time.strftime("%Y-%m-%d %H:%M:%S"),
        "execution_time_seconds": (end_time - start_time).total_seconds(),
        "CO2_emissions_kg": emissions,
        "LossFunction": lossFunction,
        "dataset": dataset

    }

    # Agregar la nueva información del entrenamiento
    existing_data.append(training_info)

    # Escribir los datos actualizados al archivo JSON
    with open(json_file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

    # Finalmente, retornamos las emisiones de CO2
    return emissions

In [24]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/diginetica"
model_path = "../trained_models"

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [24]:
%run ../Preprocess/retailrocket_preproc.py --path $dataset_path

2434084 1721538 234838
973593 261047 124195
828902 242962 39826
794204 208264 39793
784957 207891 37209
781837 204771 37209
780685 204724 36903
780319 204358 36903
780139 204353 36856
780076 204290 36856
780044 204288 36848
780033 204277 36848
780031 204277 36847
780031 204277 36847
780031 204277 36847


C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:143: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime("%Y-%m-%d %H:%M:%S.%f")
C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:146: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime("%Y-%m-%d %H:%M:%S.%f")
C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:143: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and sched

                                             Dataset  NumEvents  NumSessions  \
0  retail_rocket\retailrocket_processed_view_full...     780031       204277   
1  retail_rocket\retailrocket_processed_view_test...      29148         8036   
2  retail_rocket\retailrocket_processed_view_trai...     750832       196234   
3  retail_rocket\retailrocket_processed_view_trai...     716989       186821   
4  retail_rocket\retailrocket_processed_view_trai...      33812         9408   

   NumItems     NumDays                   StartTime  \
0     36847  137.998819  2015-05-03 03:00:40.988000   
1     11249    6.998421  2015-09-11 03:01:15.381000   
2     36824  130.999083  2015-05-03 03:00:40.988000   
3     36725  123.999511  2015-05-03 03:00:40.988000   
4     12668    6.999157  2015-09-04 03:00:34.595000   

                      EndTime  AvgItemViews  MinSessionLength  \
0  2015-09-18 02:58:58.914000     21.169457                 2   
1  2015-09-18 02:58:58.914000      2.591164               

C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:143: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min() / 1000).strftime("%Y-%m-%d %H:%M:%S.%f")
C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:146: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max() / 1000).strftime("%Y-%m-%d %H:%M:%S.%f")
C:\Users\Juanc\RecSys-Project\RecSys-Project-2024-1\Preprocess\retailrocket_preproc.py:143: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and sched

## Use a specific setup for your dataset

In [37]:
params = setups["coveo"]["params_bprmax"]
params2 = setups["coveo"]["params_xe"]

In [36]:
train_path = os.path.join(dataset_path,"diginetica_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"diginetica_processed_view_test.tsv")

In [38]:
def create_gru4rec_pytorch_script(model_name, train_folder, train_data, test_data, model_path, loss, optim, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, n_epochs, m, eval_hidden_reset, use_correct_loss, use_correct_mask_reset):
    checkpoint_dir = f"{model_path}\\{model_name}"
    s_train_full = (
        f"python ..\\GRU4REC-pytorch\\main.py --data_folder {train_folder} "
        f"--train_data {train_data} --valid_data {test_data} --checkpoint_dir {checkpoint_dir} "
        f"--num_layers 1 --embedding_dim {layers} --hidden_size {layers} "
        f"--loss_type {'BPR-max' if loss == 'bpr-max' else 'CrossEntropy'} --final_act {final_act} "
        f"--n_epochs {n_epochs} --batch_size {batch_size} --dropout_input {dropout_p_embed} "
        f"--dropout_hidden {dropout_p_hidden} --lr {learning_rate} --momentum 0.0 "
        f"--optimizer_type {'Adagrad' if optim == 'adagrad' else ''}"
        f"{' --eval_hidden_reset' if eval_hidden_reset else ''}"
        f"{' --use_correct_loss' if use_correct_loss else ''}"
        f"{' --use_correct_mask_reset' if use_correct_mask_reset else ''}"
    )
    s_test_full = s_train_full + f" --is_eval --load_model {checkpoint_dir}\\model_0000{n_epochs-1}.pt --m {m}"
    return s_train_full, s_test_full

In [39]:
# Obtenemos la configuración de los parámetros para el entrenamiento
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

In [12]:
# Obtenemos la configuración de los parámetros para el entrenamiento
loss2 = params2["loss"]
optim2 = params2["optim"]
const_emb2 = params2["constrained_embedding"]
embed2 = params2["embedding"]
final_act2 = params2["final_act"]
layers2 = params2["layers"]
batch_size2 = params2["batch_size"]
dropout_p_embed2 = params2["dropout_p_embed"]
dropout_p_hidden2 = params2["dropout_p_hidden"]
learning_rate2 = params2["learning_rate"]
momentum2 = params2["momentum"]
sample_alpha2 = params2["sample_alpha"]
bpreg2 = params2["bpreg"]
logq2 = params2["logq"]
hidden_act2 = params2["hidden_act"]
n_epochs2 = 5
m2 = '1 5 10 20'

## Train & test the out-of-the-box model

In [40]:
train_folder, train_data = '/'.join(train_path.split('/')[:-1]), train_path.split('/')[-1]
test_folder, test_data = '/'.join(test_path.split('/')[:-1]), test_path.split('/')[-1]

In [14]:
train_script_oob, test_script_oob = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_oob_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=False, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box model

In [19]:
emissions = track_training_C02_emissions_carbontracker(train_script_oob, "gru4rec_pytorch_oob_bprmax")
# Imprimimos las emisiones de carbono con estilo
if emissions is not None:
    display(HTML(f"<h2 style='color: green;'>Emisiones de CO2: {emissions} kg</h2>"))
else:
    display(HTML("<h2 style='color: red;'>Hubo un error durante la ejecución del comando.</h2>"))

CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4060 Laptop GPU.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 4060 Laptop GPU.


CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


Exception in thread Thread-30 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in positio

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: Average carbon intensity during training was 374.46 gCO2/kWh at detected location: Santiago, Santiago Metropolitan, CL.
CarbonTracker: 
Actual consumption for 1 epoch(s):
	Time:	0:04:49
	Energy:	0.006334494184 kWh
	CO2eq:	2.372035152648 g
	This is equivalent to:
	0.022065443280 km travelled by car
CarbonTracker: 
Actual consumption for 1 epoch(s):
	Time:	0:04:49
	Energy:	0.006334494184 kWh
	CO2eq:	2.372035152648 g
	This is equivalent to:
	0.022065443280 km travelled by car
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTra

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 374.46323 gCO2/kWh.


In [103]:
emissions = track_training_C02_emissions(train_script_oob, "gru4rec_pytorch_oob_bprmax")
# Imprimimos las emisiones de carbono con estilo
if emissions is not None:
    display(HTML(f"<h2 style='color: green;'>Emisiones de CO2: {emissions} kg</h2>"))
else:
    display(HTML("<h2 style='color: red;'>Hubo un error durante la ejecución del comando.</h2>"))

[codecarbon WARNING @ 17:00:48] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:00:48] [setup] RAM Tracking...
[codecarbon INFO @ 17:00:48] [setup] GPU Tracking...
[codecarbon INFO @ 17:00:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:00:49] [setup] CPU Tracking...
[codecarbon WARNING @ 17:00:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:00:51] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700HX but we don't know it. Please contact us.
[codecarbon INFO @ 17:00:51] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700HX
[codecarbon INFO @ 17:00:51] >>> Tracker's metadata:
[codecarbon INFO @ 17:00:51]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 17:00:51]   Python version: 3.12.3
[codecarbon INFO @ 17:00:51]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 17:00:51]   Available RAM : 31.701 GB
[codecarbon INFO @ 17:00:51]   CPU count: 24
[codecarbon 

Salida de STDOUT: Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                               [20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum   

### Test the out-of-the-box model

In [81]:
import subprocess

try:
    # Ejecutar el comando
    result = subprocess.run(test_script_oob, shell=True, capture_output=True, text=True)
    # Imprimir la salida estándar
    print(result.stdout)

    # Imprimir el código de retorno
    print(result.returncode)
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                     [1, 5, 10, 20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum                     

## Train & test inference fix model

In [82]:
train_script_inffix, test_script_inffix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_inffix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box eval fix model

In [92]:
emissions = track_training_C02_emissions(train_script_inffix, "gru4rec_pytorch_inffix_bprmax")
if emissions is not None:
    display(HTML(f"<h2 style='color: green;'>Emisiones de CO2: {emissions} kg</h2>"))
else:
    display(HTML("<h2 style='color: red;'>Hubo un error durante la ejecución del comando.</h2>"))

[codecarbon WARNING @ 16:21:39] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:21:39] [setup] RAM Tracking...
[codecarbon INFO @ 16:21:39] [setup] GPU Tracking...
[codecarbon INFO @ 16:21:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:21:40] [setup] CPU Tracking...
[codecarbon WARNING @ 16:21:40] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:21:42] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700HX but we don't know it. Please contact us.
[codecarbon INFO @ 16:21:42] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700HX
[codecarbon WARNING @ 16:21:42] Failed to retrieve gpu information
Traceback (most recent call last):
  File "c:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\codecarbon\core\gpu.py", line 238, in get_gpu_details
    devices_info.append(gpu_device.get_gpu_details())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "

Salida de STDOUT: Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                               [20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum   

### Test the out-of-the-box eval fix model

In [75]:
import subprocess

try:
    # Ejecutar el comando
    result = subprocess.run(test_script_inffix, shell=True, capture_output=True, text=True)
    # Imprimir la salida estándar
    print(result.stdout)
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                     [1, 5, 10, 20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum                     

## Train & test the major fix model

In [41]:
train_script_majorfix_bpr, test_script_majorfix_bpr = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_majorfix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=True, use_correct_mask_reset=True)

In [42]:
train_script_majorfix_xl, test_script_majorfix_xl = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_majorfix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss2, optim=optim2, final_act=final_act2, layers=layers2, batch_size=batch_size2, dropout_p_embed=dropout_p_embed2, dropout_p_hidden=dropout_p_hidden2, learning_rate=learning_rate2, n_epochs=n_epochs2, m=m2, eval_hidden_reset=True, use_correct_loss=True, use_correct_mask_reset=True)

In [45]:
print(train_script_majorfix_bpr)
print(test_script_majorfix_bpr)

python ..\GRU4REC-pytorch\main.py --data_folder ../datasets --train_data diginetica\diginetica_processed_view_train_full.tsv --valid_data diginetica\diginetica_processed_view_test.tsv --checkpoint_dir ../trained_models\gru4rec_pytorch_majorfix_bprmax --num_layers 1 --embedding_dim 512 --hidden_size 512 --loss_type BPR-max --final_act elu-1 --n_epochs 5 --batch_size 144 --dropout_input 0.35 --dropout_hidden 0.0 --lr 0.05 --momentum 0.0 --optimizer_type Adagrad --eval_hidden_reset --use_correct_loss --use_correct_mask_reset
python ..\GRU4REC-pytorch\main.py --data_folder ../datasets --train_data diginetica\diginetica_processed_view_train_full.tsv --valid_data diginetica\diginetica_processed_view_test.tsv --checkpoint_dir ../trained_models\gru4rec_pytorch_majorfix_bprmax --num_layers 1 --embedding_dim 512 --hidden_size 512 --loss_type BPR-max --final_act elu-1 --n_epochs 5 --batch_size 144 --dropout_input 0.35 --dropout_hidden 0.0 --lr 0.05 --momentum 0.0 --optimizer_type Adagrad --eval_h

### Train the major fix model

In [44]:
for i in range(1):
    emissions = track_training_C02_emissions(train_script_majorfix_bpr, "gru4rec_pytorch_majorfix_bprmax", "BPR", "diginetica")
    if emissions is not None:
        display(HTML(f"<h2 style='color: green;'>Emisiones de CO2: {emissions} kg</h2>"))
    else:
        display(HTML("<h2 style='color: red;'>Hubo un error durante la ejecución del comando.</h2>"))

[codecarbon WARNING @ 14:52:17] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 14:52:17] [setup] RAM Tracking...
[codecarbon INFO @ 14:52:17] [setup] GPU Tracking...
[codecarbon INFO @ 14:52:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:52:17] [setup] CPU Tracking...
[codecarbon WARNING @ 14:52:17] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 14:52:19] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700HX but we don't know it. Please contact us.
[codecarbon INFO @ 14:52:19] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700HX
[codecarbon INFO @ 14:52:19] >>> Tracker's metadata:
[codecarbon INFO @ 14:52:19]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 14:52:19]   Python version: 3.12.3
[codecarbon INFO @ 14:52:19]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 14:52:19]   Available RAM : 29.701 GB
[codecarbon INFO @ 14:52:19]   CPU count: 24
[codecarbon 

Salida de STDOUT: Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                               0.35
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                               [20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum   

### Test the major fix model

In [46]:
import subprocess

try:
    # Ejecutar el comando
    result = subprocess.run(test_script_majorfix_bpr, shell=True, capture_output=True, text=True)
    # Imprimir la salida estándar
    print(result.stdout)
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Using CUDA: True
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                               0.35
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                     [1, 5, 10, 20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum                     